# 最佳化決策模式設計與應用 
# DESIGN AND APPLICATIONS OF OPTIMAL DECISION MAKING MODELS

## Project 1 
### 李艾霓 / H24076095 / 國立成功大學 / 統計學系111級  

#### 2019/06/27

* **1 問題描述及假設**
* **2 參數設定**
    * 2.1 載入預設參數
    * 2.2 參數說明
    * 2.3 寫入參數
* **3 模型**
    * 3.1 建立模型
    * 3.2 設定變數
    * 3.3 設定限制式
    * 3.4 設定目標式
    * 3.5 最佳化
* **4 檢視結果及分析**
    * 4.1 最佳值
    * 4.2 每日購買籃數
    * 4.3 供給及需求的關係
    * 4.4 庫存
* **5 更改參數並重新測試**
    * 5.1 更改後參數
    * 5.2 放入模型
    * 5.3 結果分析
* **6 Discussion**




## 1. 問題描述及假設
<img src="img/description1.png" width=550 align="left">
<img src="img/description2.png" width=550 align="left">

須注意的是，在這裡，我們更進一步定義「耗損率」發生在每日閉店後要計算剩餘螃蟹時發現，故在庫存之前即先淘汰掉，並不會計入庫存，也不影響當日販售(當日購買的螃蟹應是經過挑選，理應不會馬上耗損，故閉店之後才計算)。

In [1]:
from gurobipy import*
import pandas as pd

## 2. 參數設定
### 2.1 載入預設參數
這是一個假想的7天需求情況:

In [2]:
demand = pd.read_csv('demand.csv', index_col=0)
demand

,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7
品種,,,,,,,
沙母,50,82,10,17,34,13,71
一般紅蟳,70,46,24,46,45,47,36
頂級沙幼母,16,62,34,44,13,45,32
二級沙幼母,20,53,65,100,81,21,51


假想的初始庫存量:

In [3]:
inventory = pd.read_csv('inventory.csv', index_col=0)
inventory

,1天庫存,2天庫存
品種,,
沙母,3,0
一般紅蟳,20,17
頂級沙幼母,13,4
二級沙幼母,21,0


而這些則是參考老師ppt所使用的參數:

In [4]:
param = pd.read_csv('parameters.csv', index_col=0)
param

,販售價格,進貨成本,庫存成本/天,每個籃子可裝,耗損率
品種,,,,,
沙母,900,700,10,6,0.05
一般紅蟳,380,300,10,20,0.03
頂級沙幼母,650,520,10,13,0.04
二級沙幼母,550,480,10,13,0.06


### 2.2 參數說明
<img src="img/ppt.png" width=550 align='left'>

此外，定義
**Ri 每個品種 i 的耗損率**

### 2.3 寫入參數

In [6]:
I = range(1,5)
J = range(3)
T = range(1,8)

In [6]:
D = {}
for i in I:
    for t in T:
        D[i,t] = demand.iat[i-1,t-1]
D

{(1, 1): 50,
 (1, 2): 82,
 (1, 3): 10,
 (1, 4): 17,
 (1, 5): 34,
 (1, 6): 13,
 (1, 7): 71,
 (2, 1): 70,
 (2, 2): 46,
 (2, 3): 24,
 (2, 4): 46,
 (2, 5): 45,
 (2, 6): 47,
 (2, 7): 36,
 (3, 1): 16,
 (3, 2): 62,
 (3, 3): 34,
 (3, 4): 44,
 (3, 5): 13,
 (3, 6): 45,
 (3, 7): 32,
 (4, 1): 20,
 (4, 2): 53,
 (4, 3): 65,
 (4, 4): 100,
 (4, 5): 81,
 (4, 6): 21,
 (4, 7): 51}

In [7]:
IC = {}
P = {}
C = {}
Q = {}
R = {}
for i in I:
    IC[i] = param.iat[i-1,2]
    P[i] = param.iat[i-1,0]
    C[i] = param.iat[i-1,1]
    Q[i] = param.iat[i-1,3]
    R[i] = param.iat[i-1,4]

In [8]:
IC

{1: 10, 2: 10, 3: 10, 4: 10}

In [9]:
P

{1: 900, 2: 380, 3: 650, 4: 550}

In [10]:
C

{1: 700, 2: 300, 3: 520, 4: 480}

In [11]:
Q

{1: 6, 2: 20, 3: 13, 4: 13}

In [12]:
R

{1: 0.05, 2: 0.03, 3: 0.04, 4: 0.06}

## 3. 模型
### 3.1 建立模型

In [13]:
model = Model('Crab')

Academic license - for non-commercial use only


### 3.2 設定變數
變數 **x [i, t]** 表示第 i 種螃蟹在第 t 天要購買的籃數。

變數 **sell [i, t, j]** 則表示第 i 種螃蟹在第 t 日販售出第 j 天庫存的數量。

In [14]:
x = {}
sell = {}
for i in I:
    for t in T:
        x[i,t] = model.addVar(ub=10, vtype='I', name='x%d%d'%(i,t))
        for j in J:
            sell[i,t,j] = model.addVar(vtype='I', name='sell%d%d%d'%(i,t,j))

由上面的變數經過轉換以方便後續運算:

**buy [i, t]** 表示第 i 種螃蟹在第 t 天實際購買隻數 (籃數 * 籃子可裝)

In [15]:
buy = {}
for i in I:
    for t in T:
        buy[i,t] = x[i,t]*Q[i]

**begin [i, t, j]** 表示第 i 種螃蟹在第 t 天<font color='red'>批貨結束之後、開店之前</font>計算出第 j 天存貨可販賣的隻數 (j=0表示當日清晨購買)

**end [i, t, j]** 表示第 i 種螃蟹在第 t 天<font color='red'>閉店之後</font>計算出第 j 天未賣完需庫存的隻數 (已扣除耗損隻數，未四捨五入，故可能不是整數，而是作為一種「期望值」)

In [16]:
begin = {}
end = {}
for i in I:
    for t in T:
        for j in range(2,-1,-1):
            if t == 1 and j in [1,2]:  # 第一天的1天及2天庫存已給定
                begin[i,1,j] = inventory.iat[i-1,j-1]
                    
            elif j != 0:  # j==2 or j==3
                begin[i,t,j] = end[i,t-1,j-1]
            else:  # j==0
                begin[i,t,j] = buy[i,t]
            
            end[i,t,j] = (begin[i,t,j]-sell[i,t,j])*(1-R[i])

In [17]:
model.update()

### 3.3 設定限制式
這裡將所有限制式分為三個部分：

In [18]:
# 一天不可買超過十籃
for t in T:
    model.addConstr(quicksum(x[i,t] for i in I) <= 10)

In [19]:
# 銷售量不可能超過當日需求量
for (i,t) in D.keys():
    model.addConstr(quicksum(sell[i,t,j] for j in J) - D[i,t] <= 0)

In [20]:
# 銷售量不可超過可販售的數量
for i in I:
    for t in T:
        for j in J:
            model.addConstr(sell[i,t,j] - begin[i,t,j] <= 0)

### 3.4 設定目標式
我們的目標式是使獲利最大，為了易讀性，我們分別算出銷售額以及成本，再以 **profit = earn-cost** 表達。

In [21]:
earn = quicksum(sell[i,t,j]*P[i] for (i,t,j) in sell)

In [22]:
cost = quicksum(buy[i,t]*C[i] for (i,t) in D.keys())  # 買進成本(當日買進個數 * 單位買進成本)
cost += quicksum(end[i,t,j]*IC[i] for (i,t,j) in sell)  # 庫存成本

In [23]:
profit = earn-cost
model.setObjective(profit, GRB.MAXIMIZE)

### 3.5 最佳化

In [24]:
model.optimize()

Optimize a model with 119 rows, 112 columns and 336 nonzeros
Variable types: 0 continuous, 112 integer (0 binary)
Coefficient statistics:
  Matrix range     [9e-01, 2e+01]
  Objective range  [4e+02, 7e+03]
  Bounds range     [1e+01, 1e+01]
  RHS range        [3e+00, 1e+02]
Found heuristic solution: objective 38132.164000
Presolve removed 52 rows and 2 columns
Presolve time: 0.00s
Presolved: 67 rows, 110 columns, 218 nonzeros
Variable types: 0 continuous, 110 integer (0 binary)

Root relaxation: objective 1.346306e+05, 80 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 134630.572    0   33 38132.1640 134630.572   253%     -    0s
H    0     0                    120728.13476 134630.572  11.5%     -    0s
     0     0 132560.557    0   41 120728.135 132560.557  9.80%     -    0s
H    0     0                    126070.36192 132560.557  5.15%     -   

## 4. 檢視結果
### 4.1 最佳值

In [25]:
print("Optimal value:", model.ObjVal)

Optimal value: 130047.64348999996


### 4.2 每日購買籃數

In [26]:
x_list = []
for i in I:
    x_sub = []
    for t in T:
        x_sub.append(int(x[i,t].X))
    x_list.append(x_sub)

x_df = pd.DataFrame(x_list,index=demand.index,columns=demand.columns)
x_df

,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7
品種,,,,,,,
沙母,7,5,1,2,5,2,8
一般紅蟳,3,1,1,2,2,3,1
頂級沙幼母,0,4,3,3,1,5,1
二級沙幼母,0,0,5,3,2,0,0


### 4.3 供給及需求的關係
recall一開始的需求表：

In [27]:
demand

,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7
品種,,,,,,,
沙母,50,82,10,17,34,13,71
一般紅蟳,70,46,24,46,45,47,36
頂級沙幼母,16,62,34,44,13,45,32
二級沙幼母,20,53,65,100,81,21,51


再對照實際賣出量：

In [28]:
supply = x_df
for i in I:
    for t in T:
        supply.iat[i-1,t-1] = sum(sell[i,t,j].X for j in range(2))
supply

,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7
品種,,,,,,,
沙母,45,30,6,12,30,12,48
一般紅蟳,53,46,20,40,40,47,32
頂級沙幼母,13,52,34,43,13,45,32
二級沙幼母,20,0,65,39,26,0,0


**將每日的需求量減去賣出量：**

In [29]:
demand - supply

,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7
品種,,,,,,,
沙母,5,52,4,5,4,1,23
一般紅蟳,17,0,4,6,5,0,4
頂級沙幼母,3,10,0,1,0,0,0
二級沙幼母,0,53,0,61,55,21,51


　　由上表可以發現，若盡量滿足頂級沙幼母的需求則可使獲利最大化。而一般紅蟳每籃獲利與頂級沙幼母相近，因此紅蟳的賣出量也與需求量很接近。
另外也能發現，為了使獲利最大化，會選擇讓沙母與二級沙幼母供不應求。

　　但在正常情況下，若有如此大量的需求，店家理應會增加批貨的量來盡量滿足需求並增加自己的獲利，故可知我們所預設的需求量應是大於實際的，可以調整我們預設的參數再測試看看。另，即使在大量的需求下，模型也會盡量減少庫存(且幾乎沒有1天和2天庫存)(見下方)，對照實際的庫存量也可以發現我們預設的初始庫存應該也是過高的。

### 4.4 庫存

In [48]:
buy = {}
begin = {}
end = {}
for i in I:
    for t in T:
        buy[i,t] = x[i,t].X*Q[i]
        for j in range(2,-1,-1):
            if t == 1 and j in [1,2]:  # 第一天的1天及2天庫存已給定
                begin[i,1,j] = inventory.iat[i-1,j-1]
                    
            elif j != 0:  # j==2 or j==3
                begin[i,t,j] = end[i,t-1,j-1]
            else:  # j==0
                begin[i,t,j] = buy[i,t]
            
            end[i,t,j] = (begin[i,t,j]-sell[i,t,j].X)*(1-R[i])

for i in I:
    print('\n%5s %3s  0天庫存  1天庫存  2天庫存'%('品種','天'))
    for t in T:
        print('%5d %5d %7.2f %7.2f %7.2f'%(i, t, end[i,t,0],end[i,t,1],end[i,t,2]))


   品種   天  0天庫存  1天庫存  2天庫存
    1     1    0.00    0.00    0.00
    1     2    0.00    0.00    0.00
    1     3    0.00    0.00    0.00
    1     4    0.00    0.00    0.00
    1     5    0.00    0.00    0.00
    1     6    0.00    0.00    0.00
    1     7    0.00    0.00    0.00

   品種   天  0天庫存  1天庫存  2天庫存
    2     1   26.19    0.00    0.00
    2     2    0.00    0.18    0.00
    2     3    0.00    0.00    0.18
    2     4    0.00    0.00    0.00
    2     5    0.00    0.00    0.00
    2     6   12.61    0.00    0.00
    2     7    0.00    0.59    0.00

   品種   天  0天庫存  1天庫存  2天庫存
    3     1    0.00    0.00    0.96
    3     2    0.00    0.00    0.00
    3     3    4.80    0.00    0.00
    3     4    0.00    0.77    0.00
    3     5    0.00    0.00    0.74
    3     6   19.20    0.00    0.00
    3     7    0.00    0.19    0.00

   品種   天  0天庫存  1天庫存  2天庫存
    4     1    0.00    0.94    0.00
    4     2    0.00    0.00    0.88
    4     3    0.00    0.00    0.00
    4     4    0.00 

## 5. 更改參數並重新測試
### 5.1 更改後參數
我們大致上將需求減少，看看是否能更符合真實情況。

In [2]:
demand = pd.read_csv('test/demand.csv', index_col=0)
demand

,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7
品種,,,,,,,
沙母,10,37,43,18,23,14,12
一般紅蟳,34,27,20,21,62,23,9
頂級沙幼母,8,12,10,12,34,17,17
二級沙幼母,6,34,23,51,12,13,11


庫存也做了調整

In [3]:
inventory = pd.read_csv('test/inventory.csv', index_col=0)
inventory

,1天庫存,2天庫存
品種,,
沙母,2,0
一般紅蟳,17,0
頂級沙幼母,10,0
二級沙幼母,1,0


下面則只更改了二級沙幼母的耗損率(0.06 -> 0.02)。

In [4]:
param = pd.read_csv('test/parameters.csv', index_col=0)
param

,販售價格,進貨成本,庫存成本/天,每個籃子可裝,耗損率
品種,,,,,
沙母,900,700,10,6,0.05
一般紅蟳,380,300,10,20,0.03
頂級沙幼母,650,520,10,13,0.04
二級沙幼母,550,480,10,13,0.02


### 5.2 放入模型

In [7]:
D = {}
for i in I:
    for t in T:
        D[i,t] = demand.iat[i-1,t-1]
IC = {}
P = {}
C = {}
Q = {}
R = {}
for i in I:
    IC[i] = param.iat[i-1,2]
    P[i] = param.iat[i-1,0]
    C[i] = param.iat[i-1,1]
    Q[i] = param.iat[i-1,3]
    R[i] = param.iat[i-1,4]
model = Model('Crab')
x = {}
sell = {}
for i in I:
    for t in T:
        x[i,t] = model.addVar(ub=10, vtype='I', name='x%d%d'%(i,t))
        for j in J:
            sell[i,t,j] = model.addVar(vtype='I', name='sell%d%d%d'%(i,t,j))
buy = {}
for i in I:
    for t in T:
        buy[i,t] = x[i,t]*Q[i]
begin = {}
end = {}
for i in I:
    for t in T:
        for j in range(2,-1,-1):
            if t == 1 and j in [1,2]:  # 第一天的1天及2天庫存已給定
                begin[i,1,j] = inventory.iat[i-1,j-1]
                    
            elif j != 0:  # j==2 or j==3
                begin[i,t,j] = end[i,t-1,j-1]
            else:  # j==0
                begin[i,t,j] = buy[i,t]
            
            end[i,t,j] = (begin[i,t,j]-sell[i,t,j])*(1-R[i])
model.update()
# 一天不可買超過十籃
for t in T:
    model.addConstr(quicksum(x[i,t] for i in I) <= 10)
# 銷售量不可能超過當日需求量
for (i,t) in D.keys():
    model.addConstr(quicksum(sell[i,t,j] for j in J) - D[i,t] <= 0)
# 銷售量不可超過可販售的數量
for i in I:
    for t in T:
        for j in J:
            model.addConstr(sell[i,t,j] - begin[i,t,j] <= 0)
earn = quicksum(sell[i,t,j]*P[i] for (i,t,j) in sell)
cost = quicksum(buy[i,t]*C[i] for (i,t) in D.keys())  # 買進成本(當日買進個數 * 單位買進成本)
cost += quicksum(end[i,t,j]*IC[i] for (i,t,j) in sell)  # 庫存成本
profit = earn-cost
model.setObjective(profit, GRB.MAXIMIZE)
model.optimize()

Academic license - for non-commercial use only
Optimize a model with 119 rows, 112 columns and 336 nonzeros
Variable types: 0 continuous, 112 integer (0 binary)
Coefficient statistics:
  Matrix range     [9e-01, 2e+01]
  Objective range  [4e+02, 7e+03]
  Bounds range     [1e+01, 1e+01]
  RHS range        [1e+00, 6e+01]
Found heuristic solution: objective 14631.968000
Presolve removed 48 rows and 5 columns
Presolve time: 0.00s
Presolved: 71 rows, 107 columns, 221 nonzeros
Variable types: 0 continuous, 107 integer (4 binary)

Root relaxation: objective 8.224825e+04, 73 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 82248.2528    0   35 14631.9680 82248.2528   462%     -    0s
H    0     0                    66012.644420 82248.2528  24.6%     -    0s
     0     0 81018.7114    0   48 66012.6444 81018.7114  22.7%     -    0s
     0     0 80057.3636 

 3180524 763502     cutoff   64      74886.0302 75568.0426  0.91%   4.3  325s
 3234670 772165 75354.3111   62   36 74886.0302 75563.0475  0.90%   4.3  330s
 3294400 781818     cutoff   49      74886.0302 75557.9515  0.90%   4.3  335s
 3352812 790816 75093.4349   60   26 74886.0302 75553.0730  0.89%   4.3  340s
 3409914 800012 75115.9068   53   42 74886.0302 75548.4025  0.88%   4.2  345s
 3470727 809539 74892.6237   55   37 74886.0302 75543.4864  0.88%   4.2  350s
 3530050 818655 75067.9450   52   39 74886.0302 75538.8794  0.87%   4.2  355s
 3588851 827636     cutoff   49      74886.0302 75534.3361  0.87%   4.2  360s
 3648899 837156 75228.0272   50   53 74886.0302 75529.7960  0.86%   4.2  365s
 3710822 846558 75399.5246   51   52 74886.0302 75525.2060  0.85%   4.2  370s
 3769441 855169 74967.4879   47   42 74886.0302 75520.7721  0.85%   4.2  375s
 3830101 864333     cutoff   58      74886.0302 75516.1942  0.84%   4.2  380s
 3891378 873129 75355.3498   55   39 74886.0302 75511.7197  0.84

 9159968 1444447 74984.5571   67   11 74886.0302 75270.4449  0.51%   3.7  850s
 9222066 1449845 75023.2690   72   33 74886.0302 75268.7370  0.51%   3.7  855s
 9286009 1454939 75266.9113   53   55 74886.0302 75266.9113  0.51%   3.7  860s
 9349279 1459630 75170.5395   58   28 74886.0302 75265.1707  0.51%   3.7  865s
 9411288 1464451 75058.5338   80   25 74886.0302 75263.4489  0.50%   3.7  870s
 9475614 1469415 75074.8712   68   32 74886.0302 75261.6488  0.50%   3.7  875s
 9540003 1474335 75018.4883   63   41 74886.0302 75259.9241  0.50%   3.7  880s
 9606582 1479633 75227.0297   69   24 74886.0302 75258.1681  0.50%   3.6  885s
 9670243 1483865     cutoff   70      74886.0302 75256.3838  0.49%   3.6  890s
 9733742 1487659 75018.2469   52   49 74886.0302 75254.5877  0.49%   3.6  895s
 9801387 1492745 74969.9385   56   49 74886.0302 75252.7150  0.49%   3.6  900s
 9867409 1497402 74944.8448   46   48 74886.0302 75250.9794  0.49%   3.6  905s
 9930144 1501503 74907.3959   85   22 74886.0302 752

 15075797 1766402 74921.6253   71   30 74886.0302 75142.8913  0.34%   3.4 1365s
 15135819 1768511     cutoff   56      74886.0302 75141.9345  0.34%   3.4 1370s
 15199298 1770456 75099.3526   80   24 74886.0302 75140.9306  0.34%   3.4 1375s
 15260491 1771968 75139.9537   70   30 74886.0302 75139.9537  0.34%   3.4 1380s
 15324957 1773774 75116.1628   55   50 74886.0302 75138.9104  0.34%   3.4 1385s
 15387001 1775277 75137.9458   60   40 74886.0302 75137.9458  0.34%   3.4 1390s
 15450470 1776866 75136.9622   50   64 74886.0302 75136.9622  0.34%   3.4 1395s
 15497128 1778121 75094.7808   59   33 74886.0302 75136.2142  0.33%   3.4 1400s
 15537146 1779020 74980.3103   55   49 74886.0302 75135.5969  0.33%   3.4 1405s
 15594261 1780849 75134.7459   60   20 74886.0302 75134.7459  0.33%   3.4 1410s
 15650642 1782318 75008.1135   56   41 74886.0302 75133.8681  0.33%   3.4 1415s
 15719614 1783857 74994.1676   56   32 74886.0302 75132.8291  0.33%   3.4 1420s
 15789490 1784901 75125.6617   79   19 7

 21785399 1776619 75057.0563   73   32 74886.0302 75057.1338  0.23%   3.1 1880s
 21856049 1774792 74949.8132   70   15 74886.0302 75056.3506  0.23%   3.1 1885s
 21927085 1772898 75055.5676   53   48 74886.0302 75055.5676  0.23%   3.1 1890s
 21994592 1770684 75054.8019   75   20 74886.0302 75054.8019  0.23%   3.1 1895s
 22064902 1768553 75054.0320   68   28 74886.0302 75054.0320  0.22%   3.1 1900s
 22138242 1766091 75052.7248   82   27 74886.0302 75053.2292  0.22%   3.1 1905s
 22204971 1764429 75052.5165   75   16 74886.0302 75052.5165  0.22%   3.1 1910s
 22276809 1763023     cutoff   56      74886.0302 75051.7667  0.22%   3.1 1915s
 22347136 1761429     cutoff   62      74886.0302 75051.0443  0.22%   3.1 1920s
 22416172 1759086 74982.0359   65   38 74886.0302 75050.2860  0.22%   3.1 1925s
 22488453 1756399 75047.7385   77    9 74886.0302 75049.4993  0.22%   3.1 1930s
 22556930 1753833     cutoff   66      74886.0302 75048.7470  0.22%   3.1 1935s
 22627573 1751366 75048.0007   59   44 7

 26819678 1556065     cutoff   78      74886.0302 75006.9355  0.16%   2.9 2395s
 26865572 1552546 74970.5177   76   20 74886.0302 75006.5052  0.16%   2.9 2400s
 26906786 1549566 75006.1092   71   35 74886.0302 75006.1092  0.16%   2.9 2405s
 26950963 1546395 75004.6732   65   42 74886.0302 75005.6852  0.16%   2.9 2410s
 26993255 1543448 75004.3711   66   19 74886.0302 75005.2843  0.16%   2.9 2415s
 27039163 1540278 75004.8458   66   31 74886.0302 75004.8458  0.16%   2.9 2420s
 27079900 1537070 75004.4613   78   12 74886.0302 75004.4613  0.16%   2.9 2425s
 27126189 1534111 75003.9867   72   29 74886.0302 75004.0366  0.16%   2.9 2430s
 27170773 1531178 75003.6427   82   16 74886.0302 75003.6427  0.16%   2.9 2435s
 27215286 1528360 74978.6572   86   18 74886.0302 75003.2580  0.16%   2.9 2440s
 27262046 1525137 75002.8180   70   20 74886.0302 75002.8180  0.16%   2.9 2445s
 27307373 1522210 74996.7827   56   25 74886.0302 75002.4056  0.16%   2.9 2450s
 27353652 1518959 74995.3316   53   36 7

 31219189 1160359 74965.6864   59    9 74886.0302 74965.6864  0.11%   2.8 2910s
 31256940 1156002 74965.3337   71   23 74886.0302 74965.3337  0.11%   2.8 2915s
 31296602 1151296 74964.7905   70   22 74886.0302 74964.9345  0.11%   2.8 2920s
 31333567 1146728 74964.5650   89   16 74886.0302 74964.5650  0.10%   2.8 2925s
 31370793 1142211 74964.2240   84   11 74886.0302 74964.2240  0.10%   2.8 2930s
 31405280 1138305 74963.8814   61   24 74886.0302 74963.8814  0.10%   2.8 2935s
 31444542 1133156 74963.4937   98    8 74886.0302 74963.5008  0.10%   2.8 2940s
 31480213 1128629 74963.1381   73   26 74886.0302 74963.1381  0.10%   2.8 2945s
 31515880 1123984 74954.2587   57   44 74886.0302 74962.7855  0.10%   2.8 2950s
 31554801 1119006 74962.4175   62   32 74886.0302 74962.4175  0.10%   2.8 2955s
 31591587 1114307 74962.0399   82   17 74886.0302 74962.0399  0.10%   2.8 2960s
 31628025 1109181 74961.6649   62   32 74886.0302 74961.6649  0.10%   2.8 2965s
 31664648 1104278 74961.2984   69   34 7

 35330277 360114 74911.9636   87   12 74886.0302 74911.9636  0.03%   2.8 3430s
 35375600 346226 74911.0872  104   11 74886.0302 74911.0872  0.03%   2.8 3435s
 35416879 332916 74910.2354   86   10 74886.0302 74910.2354  0.03%   2.8 3440s
 35460502 319257 74909.3305   64   26 74886.0302 74909.3305  0.03%   2.8 3445s
 35503202 305579     cutoff   85      74886.0302 74908.4129  0.03%   2.8 3450s
 35549375 290260 74907.4091   89   11 74886.0302 74907.4091  0.03%   2.8 3455s
 35592188 275910     cutoff   64      74886.0302 74906.4268  0.03%   2.8 3460s
 35638362 260380 74905.3733   61   17 74886.0302 74905.3733  0.03%   2.8 3465s
 35681914 246034 74904.3599   98   18 74886.0302 74904.3651  0.02%   2.8 3470s
 35719184 233939 74903.5342  107   12 74886.0302 74903.5342  0.02%   2.8 3475s
 35760005 218340     cutoff   51      74886.0302 74902.4782  0.02%   2.8 3480s
 35799723 203021     cutoff   74      74886.0302 74901.4516  0.02%   2.8 3485s
 35845304 185147 74900.2669  109   17 74886.0302 749

### 5.3 結果分析
相較於原本的參數，這次模型花了較長的時間才求得最佳解，且最佳值相較於原本確實隨著需求的減少而降低了。

In [8]:
print("Optimal value:", model.ObjVal)

Optimal value: 74886.02821999993


In [9]:
x_list = []
for i in I:
    x_sub = []
    for t in T:
        x_sub.append(int(x[i,t].X))
    x_list.append(x_sub)

x_df = pd.DataFrame(x_list,index=demand.index,columns=demand.columns)
x_df

,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7
品種,,,,,,,
沙母,1,6,7,4,3,2,2
一般紅蟳,1,1,1,1,3,1,0
頂級沙幼母,0,2,0,1,3,1,1
二級沙幼母,3,1,2,3,1,1,0


需求降低之後，不再是每天都買到十籃那麼多，而是隨著每日的需求而有所變動，這應該較符合這個模型的初衷：滿足需求的同時避免庫存太多而負擔螃蟹死亡的風險。

In [10]:
supply = x_df
for i in I:
    for t in T:
        supply.iat[i-1,t-1] = sum(sell[i,t,j].X for j in range(2))
demand - supply

,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7
品種,,,,,,,
沙母,2,1,1,0,0,2,0
一般紅蟳,0,7,0,1,2,3,9
頂級沙幼母,0,1,0,4,0,0,4
二級沙幼母,0,0,12,0,2,0,11


沒有每天都買到10籃，但這次幾乎所有的需求都被盡量滿足了！

In [11]:
buy = {}
begin = {}
end = {}
for i in I:
    for t in T:
        buy[i,t] = x[i,t].X*Q[i]
        for j in range(2,-1,-1):
            if t == 1 and j in [1,2]:  # 第一天的1天及2天庫存已給定
                begin[i,1,j] = inventory.iat[i-1,j-1]
                    
            elif j != 0:  # j==2 or j==3
                begin[i,t,j] = end[i,t-1,j-1]
            else:  # j==0
                begin[i,t,j] = buy[i,t]
            
            end[i,t,j] = (begin[i,t,j]-sell[i,t,j].X)*(1-R[i])

for i in I:
    print('\n%5s %3s  0天庫存  1天庫存  2天庫存'%('品種','天'))
    for t in T:
        print('%5d %5d %7.2f %7.2f %7.2f'%(i, t, end[i,t,0],end[i,t,1],end[i,t,2]))


   品種   天  0天庫存  1天庫存  2天庫存
    1     1    0.00    0.00    0.00
    1     2    0.00    0.00    0.00
    1     3    0.00    0.00    0.00
    1     4    5.70    0.00    0.00
    1     5    0.00    0.66    0.00
    1     6    0.00    0.00    0.63
    1     7    0.00    0.00    0.00

   品種   天  0天庫存  1天庫存  2天庫存
    2     1    0.00    2.91    0.00
    2     2    0.00    0.00    0.88
    2     3    0.00    0.00    0.00
    2     4    0.00    0.00    0.00
    2     5    0.00    0.00    0.00
    2     6    0.00    0.00    0.00
    2     7    0.00    0.00    0.00

   品種   天  0天庫存  1天庫存  2天庫存
    3     1    0.00    1.92    0.00
    3     2   14.40    0.00    0.88
    3     3    0.00    4.22    0.00
    3     4    4.80    0.00    0.22
    3     5    8.64    0.77    0.00
    3     6    0.00    4.45    0.74
    3     7    0.00    0.00    0.44

   品種   天  0天庫存  1天庫存  2天庫存
    4     1   33.32    0.00    0.00
    4     2    0.00   12.07    0.00
    4     3   14.70    0.00    0.07
    4     4    0.00 

可以看到庫存是被計算得很剛好的，正好在兩天之內將可以販售的螃蟹都販賣完畢，幾乎可以說是沒有任何的浪費。

## 6. Discussion

　　由於該最佳化求解是在每日各品種需求量已知下進行，但確切需求量往往不易獲得，需要利用過往經驗、顧客消費水平、競爭店家等來估算。因此應用實際狀況時，可能需求量的求得也會是個課題。另外，誠如上述，為了使獲利最大化會選擇讓一些品種供不應求，但如此一來會降低顧客回頭率，而回頭率會影響之後的需求量，因此供不應求所影響回頭率的情況也是之後需要考慮的問題。